In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
output = model.invoke("hi")
print(output.content)

Hi there! How can I help you today?


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model="BAAI/bge-small-en")
len(embeddings.embed_query("hi"))

c:\Users\SRJ\SRJ\Work\agentic_ai\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


384

In [3]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import operator
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage

In [4]:
loader = DirectoryLoader("./data", glob="./*.txt", loader_cls=TextLoader)

In [5]:
docs = loader.load()

In [ ]:
docs

[Document(metadata={'source': 'data\\data.txt'}, page_content="As of mid-2025, the Indian economy presents a picture of robust growth and macroeconomic stability, positioning it as one of the fastest-growing major economies in the world. Here is a snapshot of key data points and economic indicators:\n\n### Key Economic Indicators\n\n| Indicator | Value/Rate | Period | Notes |\n| :--- | :--- | :--- | :--- |\n| **GDP Growth (Real)** | **~6.5%** | FY 2024-25 | Projections for FY 2025-26 range from 6.3% to 6.8% by various agencies. |\n| **Nominal GDP** | **₹331.03 lakh crore** | FY 2024-25 | Approximately a threefold increase from ₹106.57 lakh crore in FY 2014-15. |\n| **Retail Inflation (CPI)** | **1.55%** | July 2025 | An eight-year low, primarily due to a contraction in food prices. |\n| **Fiscal Deficit** | **4.8% of GDP** | FY 2024-25 | The government has set a target of 4.4% for the financial year 2025-26. |\n| **Foreign Exchange Reserves**| **~$693.6 billion** | As of August 8, 2025

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)

In [8]:
new_docs = text_splitter.split_documents(documents=docs)

In [9]:
db = Chroma.from_documents(new_docs, embeddings)

In [10]:
retriever = db.as_retriever(search_kwargs={"k": 3})


In [11]:
retriever.invoke("growth of india")

[Document(metadata={'source': 'data\\data.txt'}, page_content='As of mid-2025, the Indian economy presents a picture of robust growth and macroeconomic stability, positioning it as one of the fastest-growing major economies in the world. Here is a snapshot of key'),
 Document(metadata={'source': 'data\\data.txt'}, page_content='* **Foreign Direct Investment (FDI):** Cumulative FDI inflows have crossed the **$1.05 trillion** mark, indicating strong global confidence in the Indian economy. The services sector is the largest'),
 Document(metadata={'source': 'data\\data.txt'}, page_content='economic growth and a major recipient of Foreign Direct Investment (FDI).')]

In [ ]:
from pydantic import BaseModel, Field

In [13]:
class Topic(BaseModel):
    Topic:str = Field(description="The topic of the question")
    Reasoning:str = Field(description="The reasoning behind topic selection")

In [14]:
from langchain.output_parsers import PydanticOutputParser

In [15]:
parser = PydanticOutputParser(pydantic_object=Topic)

In [16]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [17]:
from langchain import PromptTemplate

def function_1(state: AgentState):
    question = state["messages"][-1]
    print("Question", question)

    template = """
    Your task is to classify the user query into one of the following categories: [India, Not Related].
    Only respond with the category name and nothing else.

    User query: {question}
    {format_instructions}
    """

    prompt = PromptTemplate(
        template = template,
        input_variable = ["question"],
        partial_variables = {"format_instructions": parser.get_format_instructions()}
    )

    chain = prompt | model | parser

    response = chain.invoke({"question" : question})
    print("Parsed response:", response)
    return {"messages" : [response.Topic]}

In [18]:
def router(state: AgentState):
    print("->ROUTER->")

    last_message = state["messages"][-1]
    print("Last message:", last_message)

    if "india" in last_message.lower():
        return "RAG Call"
    else:
        return "LLM Call"

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def function_2(state: AgentState):
    print("->Function_2->")

    question = state["messages"][0]

    prompt = PromptTemplate(
        template="""
        You are a helpful assistant. Answer the question based on the context provided.
        
        Context:
        {context}
        
        Question: {question}
        """,
        input_variables=["context", "question"]
    )

    rag_chain = (
        {"context" : retriever | format_docs, "question" : RunnablePassthrough()}
        |prompt
        |model
        |StrOutputParser()
    )

    result = rag_chain.invoke(question)
    return {"messages" : [result]}

In [20]:
def function_3(state: AgentState):
    print("->LLM Call->")
    question = state["messages"][0]

    query = "Answer the question : " + question
    respose = model.invoke(query)
    return {"messages" : [respose.content]}

In [21]:
from langgraph.graph import StateGraph, END

In [22]:
workflow = StateGraph(AgentState)

In [23]:
workflow.add_node("Supervisor", function_1)

In [24]:
workflow.add_node("RAG", function_2)

In [25]:
workflow.add_node("LLM", function_3)

In [26]:
workflow.set_entry_point("Supervisor")

In [27]:
workflow.add_conditional_edges(
    "Supervisor",
    router,
    {
        "RAG Call":"RAG",
        "LLM Call":"LLM",
    }
)

In [28]:
workflow.add_edge("RAG", END)
workflow.add_edge("LLM", END)

In [29]:
app = workflow.compile()

In [30]:
state = {"messages": ["What is the growth of India?"]}
res = app.invoke(state)
print(res)

Question What is the growth of India?
Parsed response: Topic='India' Reasoning="The user query explicitly asks about 'India', making it directly related to the 'India' category."
->ROUTER->
Last message: India
->Function_2->
{'messages': ['What is the growth of India?', 'India', 'Based on the context provided:\n\n*   The Indian economy presents a picture of **robust growth** and macroeconomic stability.\n*   It is positioned as one of the **fastest-growing major economies** in the world.\n*   The Nominal GDP in FY 2024-25 (₹331.03 lakh crore) shows approximately a **threefold increase** from FY 2014-15 (₹106.57 lakh crore).']}
